In [ ]:
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader,ConcatDataset
from PIL import Image
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from tqdm import tqdm
import torch
import torch.nn as nn
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r '/content/drive/MyDrive/CGIAR' '/content/CGIAR'

In [ ]:
%cd '/content/CGIAR'

/content/drive/.shortcut-targets-by-id/14-GkeHp4qnWhJOTz3ly3H4-v53tsywG9/CGIAR


In [ ]:
!unzip images/train.zip -d '/content/CGIAR/images/train/'
!unzip images/test.zip -d  '/content/CGIAR/images/test/'

Streaming output truncated to the last 5000 lines.
 extracting: /content/CGIAR/images/test/113_initial_1_2405_2405.JPG  
 extracting: /content/CGIAR/images/test/L371F01180C39S14129Rp47127.jpg  
 extracting: /content/CGIAR/images/test/L1093F03410C39S11755Rp33366.jpg  
 extracting: /content/CGIAR/images/test/L426F01825C41S14977Rp43961.jpg  
 extracting: /content/CGIAR/images/test/L355F02270C01S05302Rp13397.jpg  
 extracting: /content/CGIAR/images/test/L361F03111C01S04825Ip.jpg  
 extracting: /content/CGIAR/images/test/L1084F02364C01S02277Rp01777.jpg  
 extracting: /content/CGIAR/images/test/L331F04674C39S14126Rp44808.jpg  
 extracting: /content/CGIAR/images/test/L141F01953C01S01679Rp00663.jpg  
 extracting: /content/CGIAR/images/test/L314F02357C01S03491Rp07852.jpg  
 extracting: /content/CGIAR/images/test/L371F04445C39S12525Rp48269.jpg  
 extracting: /content/CGIAR/images/test/L1084F02394C39S13931Rp40849.jpg  
 extracting: /content/CGIAR/images/test/L342F01526C01S00374Rp01834.jpg  
 extr

In [ ]:
df = pd.read_csv('dataset/train.csv')
train,validation = train_test_split(df, test_size=0.2, stratify=df['extent'], random_state=42)
test = pd.read_csv('dataset/test.csv')

In [ ]:
import random

# Set the seed for random module
random.seed(42)
# Set the seed for NumPy
np.random.seed(42)
# Set the seed for PyTorch
torch.manual_seed(42)
# Additional steps to set seed for CUDA (if applicable)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, tabular_data,abs_path,test=False,transform=None):
        self.tabular_data = tabular_data
        self.transform = transform
        self.abs_path = abs_path
        self.test = test
        if not test:
          self.Y = tabular_data['extent']

    def __len__(self):
        return len(self.tabular_data)

    def __getitem__(self, idx):
        x_img_path = f'{self.abs_path}/{self.tabular_data.iloc[idx]["filename"]}'
        x_img = Image.open(x_img_path)
        if self.transform:
            x_img = self.transform(x_img)
        if not self.test :
            y = np.array(self.Y.iloc[idx]).astype(np.float32).reshape(1)
            return x_img, y
        return x_img

In [ ]:
transform_train = transforms.Compose(
    [
    transforms.RandomApply([transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)], p=0.25),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.25),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean= [0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    ]
    )
transform_test = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean= [0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
])

#Swish Model

In [ ]:
class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)

class BatchNorm(nn.BatchNorm2d):
    def __init__(self, num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True):
        super().__init__(num_features, eps, momentum, affine, track_running_stats)

class SwishModel(nn.Module):
    def __init__(self,IMAGE_SIZE=256):
        super(SwishModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 128, 3)
        self.bn1 = BatchNorm(128)
        self.conv2 = nn.Conv2d(128, 256, 3)
        self.bn2 = BatchNorm(256)
        self.conv3 = nn.Conv2d(256, 256, 3)
        self.bn3 = BatchNorm(256)
        self.conv4 = nn.Conv2d(256, 256, 3)
        self.bn4 = BatchNorm(256)
        self.dropout1 = nn.Dropout2d(0.3)
        self.dropout2 = nn.Dropout2d(0.25)

        self.fc1 = nn.Linear(36864, 128)
        self.dropout3 = nn.Dropout(0.20)
        self.fc2 = nn.Linear(128, 64)
        self.dropout4 = nn.Dropout(0.25)
        self.latent = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)

    def forward(self, x):
        x = F.max_pool2d(self.bn1(Swish()(self.conv1(x))), 2)
        x = F.max_pool2d(self.bn2(Swish()(self.conv2(x))), 2)
        x = self.dropout1(x)
        x = F.max_pool2d(self.bn3(Swish()(self.conv3(x))), 2)
        x = F.max_pool2d(self.bn4(Swish()(self.conv4(x))), 2)
        x = self.dropout2(x)
        x = x.view(x.size(0), -1) # Flatten layer
        x = Swish()(self.fc1(x))
        x = self.dropout3(x)
        x = Swish()(self.fc2(x))
        x = self.dropout4(x)
        latent = Swish()(self.latent(x))
        prediction = self.fc4(latent)
        return prediction,latent

#Training Swish model

In [ ]:
train_path = 'images/train'
train_dataset_orignal = CustomDataset(train,train_path, transform=transform_train)
val_dataset = CustomDataset(validation,train_path, transform=transform_test)

In [ ]:
batch_size = 32
train_loader = DataLoader(train_dataset_orignal, batch_size=batch_size, shuffle=True,num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,num_workers=2, pin_memory=True)

In [ ]:
swish_model = SwishModel().to('cuda')

for param in swish_model.parameters():
    param.requires_grad = True

criterion = nn.MSELoss().to('cuda')

In [ ]:
optimizer = torch.optim.Adam(swish_model.parameters(),lr=1e-4)

In [ ]:
# Train model
num_epochs = 5
for epoch in (pbar:=tqdm(range(num_epochs), desc="Training")):

    total_train_loss = 0
    total_val_loss = 0

    for i, (x_img,y) in enumerate(train_loader):

        x_img = x_img.to('cuda')
        y = y.to('cuda')
        y_pred, _  = swish_model(x_img)
        loss = criterion(y_pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        pbar.set_postfix({'training':f'[{i}/{len(train_loader)}]',"Loss": loss.item(), "Val Loss": 0})
        total_train_loss += loss.detach().cpu().item()
    total_train_loss/= len(train_loader)
    with torch.no_grad():
        for i, (x_img_val,y_val) in enumerate(val_loader):
            x_img_val = x_img_val.to('cuda')
            y_val = y_val.to('cuda')

            val_y_pred, _ = swish_model(x_img_val)
            val_loss = criterion(val_y_pred, y_val)

            pbar.set_postfix({'validation':f'[{i}/{len(val_loader)}]',"Loss": loss.item(), "Val Loss": val_loss.item()})
            total_val_loss += val_loss.detach().cpu().item()
        total_val_loss/=len(val_loader)
        pbar.set_postfix({"Loss": total_train_loss, "Val Loss": total_val_loss})
        torch.save(swish_model.state_dict(), f'/content/drive/MyDrive/CGIAR/cnn/swish_{epoch}.pt')
        print('')
print("\nTraining completed.")

Training:   0%|          | 0/5 [08:17<?, ?it/s, training=[79/652], Loss=220, Val Loss=0]


KeyboardInterrupt: ignored

#extraction time

In [ ]:
swish_model = SwishModel().to('cuda')
swish_model.load_state_dict(torch.load('/content/CGIAR/cnn/swish_9.pt'))

<All keys matched successfully>

In [ ]:
train_path = 'images/train'
train_dataset_orignal = CustomDataset(train,train_path, transform=transform_train)
val_dataset = CustomDataset(validation,train_path, transform=transform_test)
test_dataset = CustomDataset(test,'images/test',test=True,transform=transform_test)

In [ ]:
batch_size = 8
orignal_train_loader = DataLoader(train_dataset_orignal, batch_size=batch_size, shuffle=False,num_workers=2, pin_memory=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,num_workers=2, pin_memory=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,num_workers=2, pin_memory=False)

In [ ]:
train_with_features_orignal = train.copy()
train_with_features_orignal = train_with_features_orignal.reindex(columns=train.columns.tolist() + [f'pix{i}' for i in range(32)])

val_with_features = validation
val_with_features = val_with_features.reindex(columns=val_with_features.columns.tolist() + [f'pix{i}' for i in range(32)])

test_with_features = test
test_with_features = test_with_features.reindex(columns=test_with_features.columns.tolist() + [f'pix{i}' for i in range(32)])

In [ ]:
start = 0
for i, (x_img,_) in enumerate(orignal_train_loader):
    x_img = x_img.to('cuda')
    _,latent = swish_model(x_img)
    nd_out = latent.detach().cpu().numpy()
    end = nd_out.shape[0] + start
    train_with_features_orignal.iloc[start:end,6:] = nd_out
    start = end

In [ ]:
train_with_features_orignal.to_csv('/content/drive/MyDrive/CGIAR/full_dataset/train.csv',index=False)

In [ ]:
start = 0
for i, (x_img,_) in enumerate(val_loader):
    x_img = x_img.to('cuda')
    _,latent = swish_model(x_img)
    nd_out = latent.detach().cpu().numpy()
    end = nd_out.shape[0] + start
    val_with_features.iloc[start:end,6:] = nd_out
    start = end

In [ ]:
val_with_features.to_csv('/content/drive/MyDrive/CGIAR/full_dataset/val.csv',index=False)

In [ ]:
start = 0
for i, (x_img) in enumerate(test_loader):
    x_img = x_img.to('cuda')
    _,latent = swish_model(x_img)
    nd_out = latent.detach().cpu().numpy()
    end = nd_out.shape[0] + start
    test_with_features.iloc[start:end,5:] = nd_out
    start = end

In [ ]:
test_with_features.to_csv('/content/drive/MyDrive/CGIAR/full_dataset/test.csv',index=False)